In [13]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import bcrypt


In [14]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Analista"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anonimizado = dados_originais_df.copy()


In [15]:
df_anonimizado["nome_completo"] = list(map(
    lambda i: f"Analista {i+1}", 
    range(len(df_anonimizado))
))

#Imprime a anonimização 
print("==== Dados anonimizados ====")
display(df_anonimizado[['nome_completo']])



==== Dados anonimizados ====


,nome_completo
0,Analista 1
1,Analista 2
2,Analista 3
3,Analista 4
4,Analista 5


In [16]:
def criptografar(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))


df_anonimizado["senha"] = df_anonimizado["senha"].apply(criptografar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["senha"]])





==== Dados anonimizados ====


,senha
0,$2b$12$nb4GHm/vG.TAzIoHPGMu5ePr/V7qqRYcnuQZGjK...
1,$2b$12$8Cm59f1EnonkqetGi2YZ8.sIWPJldFk4v/Gotor...
2,$2b$12$nsoIz7Gh17n8oOmwSHMfl.wik1Yyl4p.RPcWxHG...
3,$2b$12$E4JdbKVI3DJ95OKMTBqOH.kFYTVp3MsTu5iy4hc...
4,$2b$12$rcItoHAbsgzlaTz2VVyed.yFGwuKvFzvt5hUHeT...


In [17]:
# toda vez que for rodar esse codigo, TEM QUE RODAR o load_dotenv ANTES!! TODAS AS VEZES!!
def generalizar_data(dt):
    if pd.isna(dt):
        return "Data inválida"
    
    ano = dt.year  
    if ano < 2020:
        return "Antes de 2020"
    elif ano <= 2024:
        return "Entre 2020–2024"
    else:
        return "Após 2024"

df_anonimizado["dt_contratacao"] = df_anonimizado["dt_contratacao"].apply(generalizar_data)
display(df_anonimizado[["dt_contratacao"]])




,dt_contratacao
0,Entre 2020–2024
1,Entre 2020–2024
2,Entre 2020–2024
3,Entre 2020–2024
4,Entre 2020–2024


In [18]:
def anonimizar_cargo(cargo):
   
    texto = str(cargo).strip().replace(" ", "").replace("-", "")
    return "*" * len(texto)


df_anonimizado["cargo"] = df_anonimizado["cargo"].map(anonimizar_cargo)

print("==== Dados anonimizados ====")
display(df_anonimizado[["cargo"]])


==== Dados anonimizados ====


,cargo
0,*************
1,****************
2,****************
3,****************
4,*************


In [19]:
def anonimizar_cpf(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["cpf"] = df_anonimizado["cpf"].map(anonimizar_cpf)

print("==== Dados anonimizados ====")
display(df_anonimizado[["cpf"]])

==== Dados anonimizados ====


,cpf
0,***********
1,***********
2,***********
3,***********
4,***********


In [20]:

def pseudonimizar_email(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:0] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anonimizado["email"] = df_anonimizado["email"].apply(pseudonimizar_email)

print("==== Dados anonimizados ====")
display(df_anonimizado[["email"]])


==== Dados anonimizados ====


,email
0,****@eitruck.com
1,****@eitruck.com
2,****@eitruck.com
3,****@eitruck.com
4,****@eitruck.com


In [21]:
def anonimizar_telefone(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["telefone"] = df_anonimizado["telefone"].map(anonimizar_telefone)

print("==== Dados anonimizados ====")
display(df_anonimizado[["telefone"]])


==== Dados anonimizados ====


,telefone
0,***********
1,***********
2,***********
3,***********
4,***********


In [22]:
def anonimizar_id_unidade(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["id_unidade"] = df_anonimizado["id_unidade"].map(anonimizar_id_unidade)

print("==== Dados anonimizados ====")
display(df_anonimizado[["id_unidade"]])


==== Dados anonimizados ====


,id_unidade
0,*
1,*
2,*
3,*
4,*


In [23]:
display(df_anonimizado)

,id,id_unidade,cpf,dt_contratacao,nome_completo,email,senha,cargo,telefone
0,1,*,***********,Entre 2020–2024,Analista 1,****@eitruck.com,$2b$12$nb4GHm/vG.TAzIoHPGMu5ePr/V7qqRYcnuQZGjK...,*************,***********
1,2,*,***********,Entre 2020–2024,Analista 2,****@eitruck.com,$2b$12$8Cm59f1EnonkqetGi2YZ8.sIWPJldFk4v/Gotor...,****************,***********
2,3,*,***********,Entre 2020–2024,Analista 3,****@eitruck.com,$2b$12$nsoIz7Gh17n8oOmwSHMfl.wik1Yyl4p.RPcWxHG...,****************,***********
3,4,*,***********,Entre 2020–2024,Analista 4,****@eitruck.com,$2b$12$E4JdbKVI3DJ95OKMTBqOH.kFYTVp3MsTu5iy4hc...,****************,***********
4,5,*,***********,Entre 2020–2024,Analista 5,****@eitruck.com,$2b$12$rcItoHAbsgzlaTz2VVyed.yFGwuKvFzvt5hUHeT...,*************,***********


In [66]:
dados_originais_df.to_csv('original_analista.csv')
df_anonimizado.to_csv('pseudo_analista.csv')

In [12]:
df_anonimizado.to_sql(name='analista', con=engine_saida, if_exists='replace', index=False)

5